# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook1: Trainer Naive Bayees Regression


### **Mount google drive**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls "gdrive/My Drive/Colab Notebooks/Masters Project/"

'Airline Tweets dataset'  'Sentence Polarity Dataset'
 glove.6B.300d.txt	   VMDataset


### **Load Data and Preprocess**

In [3]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/airlinecomplaint.csv")
print(df.groupby(['label']).size())
df.head()

label
0    1700
1    1700
dtype: int64


,tweet,label
0,@united UA maintenance issues strike again. Fl...,0
1,With @AirCanada taking away more @ACAltitude b...,1
2,@DudePerfect @AmericanAir that's a lot of miss...,1
3,Oh @AmericanAir we promise to try hard not to ...,1
4,@AmericanAir liked to me that I couldn't chang...,0


**Preprocessor to Remove all special characters except emoticons**

In [4]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['tweet'][19])
print(preprocessor(df['tweet'][19]))

So @AmericanAir I'm going to need you all to get this baggage thing together. First a lost bag, then a list carousel. Yeah, umm...
so americanair i'm going to need you all to get this baggage thing together first a lost bag then a list carousel yeah umm 


In [5]:
df['tweet'] = df['tweet'].apply(preprocessor)

In [6]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stemm(text):
  return ' '.join([stemmer.stem(word) for word in text.split()])

print(stemm(df['tweet'][19]))

so americanair i'm go to need you all to get thi baggag thing togeth first a lost bag then a list carousel yeah umm


In [7]:
df['tweet'] = df['tweet'].apply(stemm)

### **Seperate Into Train and Test Sets**

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test, sentiment_train, sentiment_test = train_test_split(df['tweet'], df['label'], 
                                                                      random_state=1, test_size=0.20, 
                                                                      shuffle=False)


print('Length of train set:',len(df_train),'Length of test set:',len(df_test))

Length of train set: 2720 Length of test set: 680


### **Naive Bayees Model**

**Define Tokenizer**

In [9]:
def tokenizer(text):
  return [word for word in text.split()]

**Tokenize Text**

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(strip_accents='ascii',
                        lowercase=True,
                        preprocessor=None,
                        tokenizer= tokenizer
                       )
                            
vec.fit(df['tweet'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents='ascii', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenizer at 0x7fbb1cd2b320>,
                vocabulary=None)

In [11]:
X_train=vec.transform(df_train)
Y_train=sentiment_train
X_test=vec.transform(df_test)
Y_test=sentiment_test

**Fit Naive Bayees Regressor**

In [12]:
from sklearn.naive_bayes import MultinomialNB

clf= MultinomialNB().fit(X_train, Y_train)
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**Train and Test Scores**

In [13]:
print('Train Accuracy Score:',clf.score(X_train, Y_train))
print('Test Accuracy Score:',clf.score(X_test, Y_test))

Train Accuracy Score: 0.9113970588235294
Test Accuracy Score: 0.7323529411764705


In [15]:
from sklearn import metrics

Y_pred=clf.predict(X_test)
print('F1 Score:',metrics.f1_score(Y_test,Y_pred),
      'Precision:',metrics.precision_score(Y_test,Y_pred),
      'Recall:',metrics.recall_score(Y_test,Y_pred),
      'Accuracy:',metrics.accuracy_score(Y_test,Y_pred))

F1 Score: 0.7129337539432177 Precision: 0.7847222222222222 Recall: 0.653179190751445 Accuracy: 0.7323529411764705


### **Save the Model**

In [14]:
from joblib import dump

dump(clf, '/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/clf_naivebayees.joblib') 
dump(vec.vocabulary_, '/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/vec_naivebayees.joblib') 

['/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/vec_naivebayees.joblib']